In [1]:
import pandas as pd
from math import comb
from ax.service.ax_client import AxClient
import time
import sys

sys.path.append('../../../0_helper_functions')
import sdlnano as sdl
import calculation as calc

In [2]:
# Design space size calculation


def calculate_combinations(num_variables, total_sum, step):
    steps = int(total_sum / step)
    return comb(steps + num_variables - 1, num_variables - 1)

# Define parameters
step = 0.05
total_sum = 1

# Calculate combinations for each group
count_group1 = calculate_combinations(10, total_sum, step)  # 3 Solid lipids + 3 Polymers + 3 Liquid lipids + 1 Drug
count_group2 = calculate_combinations(4, total_sum, step)   # 3 Surfactants + 1 Water

# Calculate total combinations
total_combinations = count_group1 * count_group2


print("organic design space size: ", round(count_group1/(1e6),2), 'Million')
print("aqueous design space size: ", round(count_group2/(1e3),2), 'Thousand')
print("design space size: ", round(total_combinations/(1e9),2), 'Billion')


organic design space size:  10.02 Million
aqueous design space size:  1.77 Thousand
design space size:  17.74 Billion


# Check the iteration number

In [3]:
iteration = sdl.get_iteration_number()
print("This is the iteration: #", iteration)

This is the iteration: # 2


# Check the drug

In [4]:
drug = 'ACE'
print("This is the drug:", drug)

This is the drug: ACE


# Optimizer initialization (N/A)

# Generate recommendations

In [5]:
time_start = time.time()

previous_optimizer_file_name = '../iteration_' + str(iteration-1) + '/optimizer/optimizer_load_' + str(iteration-1) + '.json'
# generate recomms
ax_client_init = AxClient.load_from_json_file(previous_optimizer_file_name)
unlabeled_trial, ax_client_design = sdl.generate_trials(ax_client=ax_client_init, num_of_trials = 16, drug = drug, bopt=1)

time_end = time.time()

print(round((time_end-time_start)/60), 'min')

/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/botorch/acquisition/multi_objective/logei.py:418: RuntimeWarning: `cache_root` is only supported for GPyTorchModels that are not MultiTask models and don't produce a TransformedPosterior. Got a model of type <class 'botorch.models.model.ModelList'>. Setting `cache_root = False`.
  NoisyExpectedHypervolumeMixin.__init__(
[INFO 08-02 15:29:15] ax.service.ax_client: Generated new trial 32 with parameters {'Drug': 84, 'SL_1': 0, 'SL_2': 74, 'SL_3': 0, 'LL_1': 0, 'LL_2': 62, 'LL_3': 66, 'P_1': 26, 'P_2': 0, 'P_3': 0, 'S_1': 0, 'S_2': 88, 'S_3': 0, 'Water': 0} using model BoTorch.
/Users/zeqing/opt/anaconda3/envs/sdlnano_plot/lib/python3.11/site-packages/ax/core/data.py:293: FutureWarning: The behavior of DataFrame concatenation with empty or all-NA entries is deprecated. In a future version, this will no longer exclude empty or all-NA columns when determining the result dtypes. To retain the old behavior, exclude the

23 min


In [7]:
# process the trails into ratios

unlabeled_trial_processed = sdl.process_trails(unlabeled_trial)
unlabeled_trial_processed.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water
count,16.000000,16.000000,16.0,16.000000,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.0,16.00000,16.000000,16.000000,16.0
mean,39.500000,0.355750,0.0,0.230062,0.0,0.027563,0.155813,0.182563,0.038687,0.009687,0.0,0.00875,0.972562,0.018687,0.0
std,4.760952,0.183926,0.0,0.125315,0.0,0.047937,0.100068,0.204158,0.047225,0.038750,0.0,0.03500,0.109750,0.074750,0.0
min,32.000000,0.188000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00000,0.561000,0.000000,0.0
25%,35.750000,0.265250,0.0,0.207000,0.0,0.000000,0.115000,0.000000,0.000000,0.000000,0.0,0.00000,1.000000,0.000000,0.0
50%,39.500000,0.341500,0.0,0.262000,0.0,0.000000,0.172000,0.165000,0.000000,0.000000,0.0,0.00000,1.000000,0.000000,0.0
75%,43.250000,0.365000,0.0,0.317500,0.0,0.039000,0.214750,0.230750,0.077750,0.000000,0.0,0.00000,1.000000,0.000000,0.0
max,47.000000,1.000000,0.0,0.356000,0.0,0.141000,0.337000,0.632000,0.118000,0.155000,0.0,0.14000,1.000000,0.299000,0.0


In [8]:
# Convert ratios to volumes

transfer = calc.converter(unlabeled_trial_processed)
transfer.describe()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Solvent
count,16.000000,16.000000,16.0,16.000000,16.0,16.000000,16.000000,16.000000,16.000000,16.000000,16.0,16.00,16.0000,16.0000,16.0,16.00000
mean,39.500000,124.512500,0.0,80.521875,0.0,9.646875,54.534375,63.896875,13.540625,3.390625,0.0,8.75,972.5625,18.6875,0.0,-0.04375
std,4.760952,64.374062,0.0,43.860362,0.0,16.778096,35.023843,71.455220,16.528856,13.562500,0.0,35.00,109.7500,74.7500,0.0,0.17500
min,32.000000,65.800000,0.0,0.000000,0.0,0.000000,0.000000,0.000000,0.000000,0.000000,0.0,0.00,561.0000,0.0000,0.0,-0.35000
25%,35.750000,92.837500,0.0,72.450000,0.0,0.000000,40.250000,0.000000,0.000000,0.000000,0.0,0.00,1000.0000,0.0000,0.0,0.00000
50%,39.500000,119.525000,0.0,91.700000,0.0,0.000000,60.200000,57.750000,0.000000,0.000000,0.0,0.00,1000.0000,0.0000,0.0,0.00000
75%,43.250000,127.750000,0.0,111.125000,0.0,13.650000,75.162500,80.762500,27.212500,0.000000,0.0,0.00,1000.0000,0.0000,0.0,0.00000
max,47.000000,350.000000,0.0,124.600000,0.0,49.350000,117.950000,221.200000,41.300000,54.250000,0.0,140.00,1000.0000,299.0000,0.0,0.35000


In [9]:
#dead_volume = 3.0 # mL

round(transfer[['Drug','SL_1', 'SL_2', 'SL_3', 'LL_1', 'LL_2', 'LL_3', 'P_1', 'P_2', 'P_3', 'S_1', 'S_2', 'S_3', 'Water', 'Solvent']].sum()/1000,1)

Drug        2.0
SL_1        0.0
SL_2        1.3
SL_3        0.0
LL_1        0.2
LL_2        0.9
LL_3        1.0
P_1         0.2
P_2         0.1
P_3         0.0
S_1         0.1
S_2        15.6
S_3         0.3
Water       0.0
Solvent    -0.0
dtype: float64

In [10]:
# file path
ax_client_design_path = "optimizer/optimizer_design_" + str(iteration) + ".json"

unlabeled_trial_path = "data/unlabeled_" + str(iteration) + ".xlsx"

unlabeled_trial_processed_path = "data/unlabeled_processed_" + str(iteration) + ".xlsx"

transfer_path = 'data/transfer_' + str(iteration) + '.xlsx'


In [11]:
ax_client_design.save_to_json_file(ax_client_design_path)

unlabeled_trial.to_excel(unlabeled_trial_path, index=False)

unlabeled_trial_processed.to_excel(unlabeled_trial_processed_path, index=False)

transfer.to_excel(transfer_path, index = False)

# Generate new protocol

In [12]:
# Paths
template_protocol = '../../templates/OT_protocol_template.py'

output_script_path_1 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_1.py'
output_script_path_2 = 'protocol/iteration_' + str(iteration) + '_OT_2_protocol_2.py'



In [13]:
# Update the script with Excel data
updated_script = calc.update_transfer_script(template_protocol, transfer_path, output_script_path_1, output_script_path_2) 


In [14]:
updated_script

('protocol/iteration_2_OT_2_protocol_1.py',
 'protocol/iteration_2_OT_2_protocol_2.py')

# Perform experiment

# Results

In [15]:
results = calc.parameters_and_complexity(iteration)
results

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,P_3,S_1,S_2,S_3,Water,Complexity,Complexity_STD
0,32,0.269,0,0.237,0,0.000,0.199,0.212,0.083,0.000,0,0.00,1.000,0.000,0,5,0
1,33,0.368,0,0.000,0,0.000,0.000,0.632,0.000,0.000,0,0.00,1.000,0.000,0,2,0
2,34,0.357,0,0.287,0,0.000,0.213,0.143,0.000,0.000,0,0.00,1.000,0.000,0,4,0
3,35,0.405,0,0.356,0,0.000,0.239,0.000,0.000,0.000,0,0.00,1.000,0.000,0,3,0
4,36,0.189,0,0.195,0,0.034,0.153,0.187,0.087,0.155,0,0.14,0.561,0.299,0,9,0
5,37,0.385,0,0.000,0,0.000,0.000,0.615,0.000,0.000,0,0.00,1.000,0.000,0,2,0
6,38,0.302,0,0.302,0,0.000,0.145,0.251,0.000,0.000,0,0.00,1.000,0.000,0,4,0
7,39,0.329,0,0.308,0,0.000,0.245,0.000,0.118,0.000,0,0.00,1.000,0.000,0,4,0
8,40,0.340,0,0.259,0,0.000,0.337,0.000,0.065,0.000,0,0.00,1.000,0.000,0,4,0
9,41,0.348,0,0.348,0,0.000,0.174,0.129,0.000,0.000,0,0.00,1.000,0.000,0,4,0


In [16]:
size_raw = calc.size_raw(iteration)
size_raw

,Data Quality,Item,Size,PD Index
0,Caution,A1,284.242166,0.284041
1,Good,A2,288.301722,0.243315
2,Caution,A3,297.440119,0.309958
3,Caution,A4,270.098728,0.460556
4,Good,A5,256.112446,0.284748
5,Caution,A6,262.753682,0.352981
6,Good,B1,271.752503,0.343767
7,Caution,B2,288.459836,0.372348
8,Good,B3,279.436029,0.307083
9,Good,B4,270.921858,0.294452


In [18]:
size_processed = calc.process_formulations(size_raw)
size_processed

,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0
1,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,Formulation 3,275.594266,5.433073,0.325425,0.025940,1
3,Formulation 4,272.696961,2.510373,0.297252,0.003959,1
4,Formulation 5,296.012364,4.088638,0.277985,0.003849,1
5,Formulation 6,255.459967,4.598726,0.378823,0.049395,1
6,Formulation 7,290.950790,2.883850,0.331629,0.056686,1
7,Formulation 8,267.418761,8.022059,0.176781,0.063436,1
8,Formulation 9,272.713814,1.036558,0.254658,0.026656,1
9,Formulation 10,275.239238,2.040534,0.249545,0.026125,1


In [19]:
results_final = pd.concat([results, size_processed], axis=1)
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,S_3,Water,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality
0,32,0.269,0,0.237,0,0.000,0.199,0.212,0.083,0.000,...,0.000,0,5,0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0
1,33,0.368,0,0.000,0,0.000,0.000,0.632,0.000,0.000,...,0.000,0,2,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0
2,34,0.357,0,0.287,0,0.000,0.213,0.143,0.000,0.000,...,0.000,0,4,0,Formulation 3,275.594266,5.433073,0.325425,0.025940,1
3,35,0.405,0,0.356,0,0.000,0.239,0.000,0.000,0.000,...,0.000,0,3,0,Formulation 4,272.696961,2.510373,0.297252,0.003959,1
4,36,0.189,0,0.195,0,0.034,0.153,0.187,0.087,0.155,...,0.299,0,9,0,Formulation 5,296.012364,4.088638,0.277985,0.003849,1
5,37,0.385,0,0.000,0,0.000,0.000,0.615,0.000,0.000,...,0.000,0,2,0,Formulation 6,255.459967,4.598726,0.378823,0.049395,1
6,38,0.302,0,0.302,0,0.000,0.145,0.251,0.000,0.000,...,0.000,0,4,0,Formulation 7,290.950790,2.883850,0.331629,0.056686,1
7,39,0.329,0,0.308,0,0.000,0.245,0.000,0.118,0.000,...,0.000,0,4,0,Formulation 8,267.418761,8.022059,0.176781,0.063436,1
8,40,0.340,0,0.259,0,0.000,0.337,0.000,0.065,0.000,...,0.000,0,4,0,Formulation 9,272.713814,1.036558,0.254658,0.026656,1
9,41,0.348,0,0.348,0,0.000,0.174,0.129,0.000,0.000,...,0.000,0,4,0,Formulation 10,275.239238,2.040534,0.249545,0.026125,1


In [20]:
# µg/mL
results_final['Solu'] = results_final['Drug'] * 600 / 0.3
results_final['Solu_STD'] = 0

results_final.loc[results_final['Formulation Quality'] == 0, 'Solu'] = 0
results_final

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,32,0.269,0,0.237,0,0.000,0.199,0.212,0.083,0.000,...,5,0,Formulation 1,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
1,33,0.368,0,0.000,0,0.000,0.000,0.632,0.000,0.000,...,2,0,Formulation 2,1000.000000,0.000000,1.000000,0.000000,0,0.0,0
2,34,0.357,0,0.287,0,0.000,0.213,0.143,0.000,0.000,...,4,0,Formulation 3,275.594266,5.433073,0.325425,0.025940,1,714.0,0
3,35,0.405,0,0.356,0,0.000,0.239,0.000,0.000,0.000,...,3,0,Formulation 4,272.696961,2.510373,0.297252,0.003959,1,810.0,0
4,36,0.189,0,0.195,0,0.034,0.153,0.187,0.087,0.155,...,9,0,Formulation 5,296.012364,4.088638,0.277985,0.003849,1,378.0,0
5,37,0.385,0,0.000,0,0.000,0.000,0.615,0.000,0.000,...,2,0,Formulation 6,255.459967,4.598726,0.378823,0.049395,1,770.0,0
6,38,0.302,0,0.302,0,0.000,0.145,0.251,0.000,0.000,...,4,0,Formulation 7,290.950790,2.883850,0.331629,0.056686,1,604.0,0
7,39,0.329,0,0.308,0,0.000,0.245,0.000,0.118,0.000,...,4,0,Formulation 8,267.418761,8.022059,0.176781,0.063436,1,658.0,0
8,40,0.340,0,0.259,0,0.000,0.337,0.000,0.065,0.000,...,4,0,Formulation 9,272.713814,1.036558,0.254658,0.026656,1,680.0,0
9,41,0.348,0,0.348,0,0.000,0.174,0.129,0.000,0.000,...,4,0,Formulation 10,275.239238,2.040534,0.249545,0.026125,1,696.0,0


In [21]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.416667
1,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.166667
2,1,0.275594,0.005433,0.357,0.0,0.325425,0.025940,0.333333
3,1,0.272697,0.002510,0.405,0.0,0.297252,0.003959,0.250000
4,1,0.296012,0.004089,0.189,0.0,0.277985,0.003849,0.750000
5,1,0.255460,0.004599,0.385,0.0,0.378823,0.049395,0.166667
6,1,0.290951,0.002884,0.302,0.0,0.331629,0.056686,0.333333
7,1,0.267419,0.008022,0.329,0.0,0.176781,0.063436,0.333333
8,1,0.272714,0.001037,0.340,0.0,0.254658,0.026656,0.333333
9,1,0.275239,0.002041,0.348,0.0,0.249545,0.026125,0.333333


In [22]:
results_normalized = sdl.normalize(results_final)
results_normalized[['Formulation Quality','Size', 'Size_STD', 'Solu', 'Solu_STD', 'PDI', 'PDI_STD','Complexity']]

,Formulation Quality,Size,Size_STD,Solu,Solu_STD,PDI,PDI_STD,Complexity
0,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.416667
1,0,1.000000,0.000000,0.000,0.0,1.000000,0.000000,0.166667
2,1,0.275594,0.005433,0.357,0.0,0.325425,0.025940,0.333333
3,1,0.272697,0.002510,0.405,0.0,0.297252,0.003959,0.250000
4,1,0.296012,0.004089,0.189,0.0,0.277985,0.003849,0.750000
5,1,0.255460,0.004599,0.385,0.0,0.378823,0.049395,0.166667
6,1,0.290951,0.002884,0.302,0.0,0.331629,0.056686,0.333333
7,1,0.267419,0.008022,0.329,0.0,0.176781,0.063436,0.333333
8,1,0.272714,0.001037,0.340,0.0,0.254658,0.026656,0.333333
9,1,0.275239,0.002041,0.348,0.0,0.249545,0.026125,0.333333


In [23]:
labeled_data_path = 'data/labeled_' + str(iteration) + '.xlsx'
labeled_norm_path = 'data/labeled_norm_' + str(iteration) + '.xlsx'


In [24]:
results_final.to_excel(labeled_data_path, index=False)
results_normalized.to_excel(labeled_norm_path, index=False)

# Load the labeled data to update the optimizer

In [25]:
labeled_norm = pd.read_excel(labeled_norm_path)
labeled_norm.head()

,trial_index,Drug,SL_1,SL_2,SL_3,LL_1,LL_2,LL_3,P_1,P_2,...,Complexity,Complexity_STD,Formulation,Size,Size_STD,PDI,PDI_STD,Formulation Quality,Solu,Solu_STD
0,32,0.269,0,0.237,0,0.000,0.199,0.212,0.083,0.000,...,0.416667,0,Formulation 1,1.000000,0.000000,1.000000,0.000000,0,0.000,0
1,33,0.368,0,0.000,0,0.000,0.000,0.632,0.000,0.000,...,0.166667,0,Formulation 2,1.000000,0.000000,1.000000,0.000000,0,0.000,0
2,34,0.357,0,0.287,0,0.000,0.213,0.143,0.000,0.000,...,0.333333,0,Formulation 3,0.275594,0.005433,0.325425,0.025940,1,0.357,0
3,35,0.405,0,0.356,0,0.000,0.239,0.000,0.000,0.000,...,0.250000,0,Formulation 4,0.272697,0.002510,0.297252,0.003959,1,0.405,0
4,36,0.189,0,0.195,0,0.034,0.153,0.187,0.087,0.155,...,0.750000,0,Formulation 5,0.296012,0.004089,0.277985,0.003849,1,0.189,0


In [26]:
ax_client_design = AxClient.load_from_json_file(ax_client_design_path)


In [27]:
loaded_optimizer_path = "optimizer/optimizer_load_" + str(iteration) + ".json"

In [28]:
loaded_ax_client = sdl.load_labeled_data(ax_client_design, labeled_norm_path)
loaded_ax_client.save_to_json_file(loaded_optimizer_path)

[INFO 08-02 17:45:53] ax.service.ax_client: Completed trial 32 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-02 17:45:53] ax.service.ax_client: Completed trial 33 with data: {'Solu': (0.0, 0), 'Size': (1.0, 0.0), 'PDI': (1.0, 0.0)}.
[INFO 08-02 17:45:53] ax.service.ax_client: Completed trial 34 with data: {'Solu': (0.357, 0), 'Size': (0.275594, 0.005433), 'PDI': (0.325425, 0.02594)}.
[INFO 08-02 17:45:53] ax.service.ax_client: Completed trial 35 with data: {'Solu': (0.405, 0), 'Size': (0.272697, 0.00251), 'PDI': (0.297252, 0.003959)}.
[INFO 08-02 17:45:53] ax.service.ax_client: Completed trial 36 with data: {'Solu': (0.189, 0), 'Size': (0.296012, 0.004089), 'PDI': (0.277985, 0.003849)}.
[INFO 08-02 17:45:53] ax.service.ax_client: Completed trial 37 with data: {'Solu': (0.385, 0), 'Size': (0.25546, 0.004599), 'PDI': (0.378823, 0.049395)}.
[INFO 08-02 17:45:53] ax.service.ax_client: Completed trial 38 with data: {'Solu': (0.302, 0), 'Size': (0.290951, 0.0